# EFM Generation: CIFAR10

In [ ]:
import torch
import torchvision


import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import wandb
import sys
sys.path.append("/trinity/home/a.kolesov/EFM/")
from src.utils import Config, optimization_manager, forward_interpolation
from src.models import DDPM, ExponentialMovingAverage
from src.efm_field import EFMGroundTruth
from src.ode import LearnedImageODESolver
from src.train import Train

## 1. Config

In [ ]:
config = Config()


config.device = 'cuda'
config.data = Config()
config.data.name='CIFAR10'
config.data.num_channels = 3
config.data.img_resize = 32
config.data.image_size=32
config.data.centered = True


config.DIM = config.data.num_channels*config.data.image_size*config.data.image_size + 1
config.L = 500


config.p = Config()
config.p.x_loc = 0.

config.q = Config()
config.q.x_loc = config.L


config.training = Config()
config.training.small_batch_size = 128
config.training.batch_size = 4096
config.training.n_iters = 1_000_000
config.training.sde = 'poisson'
config.training.eval_freq = 1_000
config.training.snapshot_freq = 5_000
config.training.sigma_end = 0.01
config.training.M = 291
config.training.tau = 0.03
config.training.epsilon = .5
config.training.interpolation = 'both_side'


config.model  = Config()
config.model = config.model
config.model.name = 'ncsnpp'
config.model.scale_by_sigma = False
config.model.ema_rate = 0.9999
config.model.normalization = 'GroupNorm'
config.model.nonlinearity = 'swish'
config.model.nf = 128
config.model.ch_mult = (1, 2, 2, 2)
config.model.num_res_blocks = 4
config.model.attn_resolutions = (16,)
config.model.resamp_with_conv = True
config.model.conditional = True
config.model.fir = False
config.model.fir_kernel = [1, 3, 3, 1]
config.model.skip_rescale = True
config.model.resblock_type = 'biggan'
config.model.progressive = 'none'
config.model.progressive_input = 'none'
config.model.progressive_combine = 'sum'
config.model.attention_type = 'ddpm'
config.model.init_scale = 0.
config.model.fourier_scale = 16
config.model.embedding_type = 'positional'
config.model.conv_size = 3
config.model.dropout = 0.1

config.optim  = Config()
config.optim.weight_decay = 0
config.optim.optimizer = 'Adam'
config.optim.lr = 2e-4
config.optim.beta1 = 0.9
config.optim.eps = 1e-8
config.optim.warmup = 5000
config.optim.grad_clip = 1.

config.efm = Config()
config.efm.training = Config()
config.efm.training.stability = True
config.efm.training.gamma = 1e-7
#config.efm.dist_behind = 9.

config.ode = Config()
config.ode.gamma = 1e-7
config.ode.step = 0.25

## 2. Data

In [ ]:
TRANSFORM = torchvision.transforms.Compose([
    torchvision.transforms.Resize(config.data.img_resize),
    torchvision.transforms.ToTensor(),
    #torchvision.transforms.Normalize((0.5, 0.5, 0.5),(0.5, 0.5, 0.5)),
    #lambda x: .5*x + .5
])


train_data = torchvision.datasets.CIFAR10(root='/trinity/home/a.kolesov/data/CIFAR10/',
                                          train=True, download=True, transform=TRANSFORM)
eval_data = torchvision.datasets.CIFAR10(root='/trinity/home/a.kolesov/data/CIFAR10/',
                                         train=False, download=True, transform=TRANSFORM)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=config.training.batch_size,
                                           shuffle=True)
eval_loader =  torch.utils.data.DataLoader(eval_data, batch_size=config.training.batch_size, 
                                           shuffle=True)

train_iter = iter(train_loader)
eval_iter = iter(eval_loader)

## 3. Models

In [ ]:
net = DDPM(config).to(config.device)
params = net.parameters()
optimizer = torch.optim.Adam(params,
                       lr=config.optim.lr, betas=(config.optim.beta1, 0.999), eps=config.optim.eps,
                       weight_decay=config.optim.weight_decay)

ema = ExponentialMovingAverage(net.parameters(), decay=config.model.ema_rate)
state = dict(optimizer=optimizer, model=net, ema=ema, step=0)
optimize_fn = optimization_manager(config)

## 4. Training

In [ ]:
config.name_exp = f"RightNoise_NonDippole_Int={config.training.interpolation}_\
            eps={config.training.epsilon}_sigma={config.training.epsilon}_\
            BS_{config.training.batch_size}_SBS={config.training.small_batch_size}\
            _L_{config.training.epsilon }"
wandb.init(project="EFMGenerationCIFAR", name=config.name_exp)

In [ ]:
net, state = Train(EFMGroundTruth, net, optimizer, optimize_fn, state, train_loader, eval_loader, config)